<center><h1>Chatbot</h1></center>

# Imports

In [77]:
import json
import nltk
import random
import numpy as np

from nltk import data
from nltk.stem.porter import PorterStemmer

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Constants

In [78]:
all_words = []
tags = []
xy = []

# Reading the Dataset

In [79]:
with open("sample.json", "r") as f:
    intents = json.load(f)
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hey',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day'],
   'responses': ['Hey :-)',
    'Hello, thanks for visiting',
    'Hi there, what can I do for you?',
    'Hi there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'items',
   'patterns': ['Which items do you have?',
    'What kinds of items are there?',
    'What do you sell?'],
   'responses': ['We sell coffee and tea', 'We have coffee and tea']},
  {'tag': 'payments',
   'patterns': ['Do you take credit cards?',
    'Do you accept Mastercard?',
    'Can I pay with Paypal?',
    'Are you cash only?'],
   'responses': ['We 

# Preprocessing

In [80]:
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    sentence_words = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype=np.float32)
    for (idx, w) in enumerate(all_words):
        if w in sentence_words:
            bag[idx] = 1.0
    return bag


In [81]:
for intent in intents["intents"]:
    tag = intent["tag"]
    tags.append(tag)
    for pattern in intent["patterns"]:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ["?", "!", "."]
all_words = [stem(w) for w in all_words if w not in ignore_words]

all_words = sorted(set(all_words))
tags = sorted(set(tags))
all_words

["'s",
 'a',
 'accept',
 'anyon',
 'are',
 'bye',
 'can',
 'card',
 'cash',
 'credit',
 'day',
 'deliveri',
 'do',
 'doe',
 'funni',
 'get',
 'good',
 'goodby',
 'have',
 'hello',
 'help',
 'hey',
 'hi',
 'how',
 'i',
 'is',
 'item',
 'joke',
 'kind',
 'know',
 'later',
 'long',
 'lot',
 'mastercard',
 'me',
 'my',
 'of',
 'onli',
 'pay',
 'paypal',
 'see',
 'sell',
 'ship',
 'someth',
 'take',
 'tell',
 'thank',
 'that',
 'there',
 'what',
 'when',
 'which',
 'with',
 'you']

In [82]:
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.]], dtype=float32)

## Hyper Parameters

In [83]:
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 200

In [84]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # dataset[idx]
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples


dataset = ChatDataset()
train_loader = DataLoader(
    dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0
)

In [85]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # out = self.relu(out)
        return out

In [86]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward
        outputs = model(words)
        loss = criterion(outputs, labels)

        # Backward and Optimizer
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}")
    if loss.item() < 0.01:
        break

print(f"Final Loss, loss={loss.item():.4f}")

data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags,
}

FILE = (f"backup/ipynbdata.pth")
torch.save(data, FILE)

print(f"training complete. file saved to {FILE}")

Epoch 1/200, loss=1.7658
Epoch 2/200, loss=1.8239
Epoch 3/200, loss=1.8327
Epoch 4/200, loss=2.1734
Epoch 5/200, loss=1.7086
Epoch 6/200, loss=2.0719
Epoch 7/200, loss=1.7031
Epoch 8/200, loss=1.8664
Epoch 9/200, loss=1.6915
Epoch 10/200, loss=1.7687
Epoch 11/200, loss=2.1112
Epoch 12/200, loss=2.1499
Epoch 13/200, loss=1.7228
Epoch 14/200, loss=1.9108
Epoch 15/200, loss=1.9359
Epoch 16/200, loss=1.5877
Epoch 17/200, loss=2.1293
Epoch 18/200, loss=2.0893
Epoch 19/200, loss=2.2980
Epoch 20/200, loss=1.9052
Epoch 21/200, loss=1.8491
Epoch 22/200, loss=1.6589
Epoch 23/200, loss=1.6620
Epoch 24/200, loss=1.9098
Epoch 25/200, loss=1.9268
Epoch 26/200, loss=1.5625
Epoch 27/200, loss=1.7993
Epoch 28/200, loss=2.0873
Epoch 29/200, loss=1.8359
Epoch 30/200, loss=1.9506
Epoch 31/200, loss=1.7697
Epoch 32/200, loss=1.6464
Epoch 33/200, loss=2.1796
Epoch 34/200, loss=1.6795
Epoch 35/200, loss=1.6057
Epoch 36/200, loss=1.6994
Epoch 37/200, loss=1.4990
Epoch 38/200, loss=1.6980
Epoch 39/200, loss=2.

# Testing

### Constants

In [87]:
FILE = "./backup/ipynbdata.pth"
data = torch.load(FILE)

### Loading the Data

In [ ]:
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

In [88]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [89]:
bot_name = "Sam"
print("Shoot a Question !")
while True:
    sentence = input("You : ")
    print(f"You : {sentence}")
    if sentence == "quit" or sentence == "exit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.50:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                print(f"{bot_name} : {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name} : I do not understand...")

Shoot a Question !
You : exit
